In [ ]:
import duckdb

In [ ]:
con = duckdb.connect(database='dados_duckdb.db', read_only=False)

In [ ]:
con.execute("SELECT * FROM bronze_z0019")

In [ ]:
df = con.execute("SELECT * FROM bronze_z0019").fetchdf()
df.head()

In [ ]:
df.columns

In [ ]:
df_final = df.drop(columns=['nome_arquivo', 'data_ingestao'])
df_final.head()


In [ ]:
df = con.execute("""
                 SELECT *
                 FROM (
                    SELECT *, ROW_NUMBER() OVER (PARTITION BY NATBR ORDER BY data_ingestao DESC) AS row
                    FROM bronze_z0019
                    WHERE data_ingestao >= '2025-01-11'
                 ) WHERE row = 1
                 """).fetchdf()
df.head(10)

In [ ]:
df.head(10)

In [ ]:
df.drop(columns=['nome_arquivo', 'data_ingestao', 'run'], inplace=True)

df.head(10)

In [ ]:
colunas_para_remover = ['nome_arquivo', 'data_ingestão','row']

In [ ]:
colunas_existentes = [col for col in
colunas_para_remover if col in df.columns]

In [ ]:
df.drop(columns=colunas_existentes, inplace=True)

In [ ]:
df.head()

In [ ]:
df_final = df.drop(columns=['nome_arquivo','data_ingestao','row'])

df_final.head(10)

In [ ]:
df_final = df.copy()

In [ ]:
df_final.head()

In [ ]:
df_final.drop(columns=['data_ingestao'],inplace=True)

In [ ]:
df_final.head()

In [ ]:
print(df_final.columns)

In [ ]:
df_final.columns = df_final.columns.str.strip()

In [ ]:
df_final.dtypes

In [ ]:
df_final = df_final.rename(columns={"NATBR":"id"})
df_final = df_final.rename(columns={"MAKTX":"nm_produto"})
df_final = df_final.rename(columns={"WERKS":"id_categoria"})
df_final = df_final.rename(columns={"MAINS":"id_fornecedor"})
df_final = df_final.rename(columns={"LABST":"vl_preco"})
df_final.head(10)

In [ ]:
df2 = df_final.astype({
        'ID': 'int32', # <- corrigido
        'nm_produto': 'string',
        'id_categoria': 'string',
        'id_fornecedor': 'int32',
        'vl_preco': 'float32'
})
df2.head(10)

In [ ]:
df_final.columns = df_final.columns.str.lower()

In [ ]:
df2 = df_final.astype({
        'id': int,
        'nm_produto': str,
        'id_categoria': str,
        'id_fornecedor': int,
        'vl_preco': float
})

df2.dtypes
#df2.head(10)

In [ ]:
con.execute("""
CREATE TABLE IF NOT EXISTS produtos (
            id BIGINT,
            nm_produto TEXT,
            id_fornecedor BIGINT,
            vl_preco FLOAT
            )
""")

In [ ]:
df2.head(10)

In [ ]:
con.execute("INSERT INTO produtos SELECT * from df2")

In [ ]:
df_resultado = con.execute("select * from produtos").fetchdf()
df_resultado.head(10)

In [ ]:
df2_filtrado = df2[['id', 'nm_produto', 'id_fornecedor', 'vl_preco']]

In [ ]:
con.execute("INSERT INTO produtos SELECT * FROM df2_filtrado")

In [ ]:
df_resultado = con.execute("SELECT * FROM produtos").fetchdf()
df_resultado.head(10)

In [ ]:
con.execute("DROP TABLE IF EXISTS produtos")

In [ ]:
con.execute("""
    CREATE TABLE produtos (
        id INTEGER,
        nm_produto STRING,
        id_categoria STRING,
        id_fornecedor INTEGER,
        vl_preco FLOAT
    )
""")

In [ ]:
con.execute("INSERT INTO produtos SELECT * FROM df2")

In [ ]:
df_resultado = con.execute("SELECT * FROM produtos").fetchdf()
df_resultado.head(10)

In [ ]:
con.close()